In [36]:
import pandas as pd

import pandas as pd
for encoding in ['utf-8', 'latin-1', 'windows-1252']:
    try:
        df = pd.read_csv('./fraudTest.csv', encoding=encoding)
        break 
    except UnicodeDecodeError:
        pass

df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['dob'] = pd.to_datetime(df['dob'])


In [70]:
selected_features = [
 'cc_num', 'trans_date_trans_time', 'merchant','amt', 
    'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 
    'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 
    'merch_lat', 'merch_long'
]


target_variable = 'is_fraud'

features = selected_df = df[selected_features]
target = selected_df = df[target_variable]

from sklearn.preprocessing import LabelEncoder

encoded_df = pd.DataFrame()

label_encoders = {}

for col in selected_features:
    label_encoders[col] = LabelEncoder()
    encoded_df[col+"_n"] = label_encoders[col].fit_transform(df[col])

print(encoded_df)

        amt_n
0         186
1        2884
2        4028
3        5905
4         219
...       ...
555714   4277
555715  11084
555716   8588
555717    699
555718   3713

[555719 rows x 1 columns]


In [71]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(encoded_df, target, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

y_pred_train = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy:", train_accuracy)

y_pred_test = clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy:", test_accuracy)


Training Accuracy: 0.998639149749761
Test Accuracy: 0.9951594328078889


In [74]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/predict_fraud', methods=['POST'])
def predict_fraud():
    predict = request.json
    data = pd.DataFrame([predict])

    data['user_id'] = label_encoders['gender'].transform([data['user_id']])[0]
    data['age'] = label_encoders['dob'].transform([data['age']])[0]
    data['bday'] = label_encoders['dob'].transform([data['bday']])[0]
    data['sex'] = label_encoders['gender'].transform([data['sex']])[0]
    data['user_blocked'] = label_encoders['sex'].transform([data['sex']])[0]
    data['user_registration_datetime'] = label_encoders['trans_date_trans_time'].transform([data['user_registration_datetime']])[0]
    data['deposit_id'] = label_encoders['sex'].transform([data['sex']])[0]
    data['transaction_date'] = label_encoders['trans_date_trans_time'].transform([data['transaction_date']])[0]
    data['type'] = label_encoders['dob'].transform([data['type']])[0]
    data['amt_n'] = label_encoders['amt'].transform([data['total_amount']])[0]
    data['in'] = label_encoders['type'].inverse_transform([data['']])[0]
    
    newdata = data.drop(["total_amount", "total_amount", "age", "bday", "sex", "user_blocked", "user_registration_datetime", "user_registration_ip", "deposit_id", "transaction_date", "type", "in","user_id" ], axis="columns")


    prediction = clf.predict(newdata)
    r = prediction[0]
    if r == 0:
        return jsonify({"answer": 'non fraudulent'})
    else:
        return jsonify({"answer": 'fraudulent'})

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
127.0.0.1 - - [06/Apr/2024 03:50:00] "POST /predict_fraud HTTP/1.1" 200 -
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
127.0.0.1 - - [06/Apr/2024 03:58:50] "POST /predict_fraud HTTP/1.1" 200 -
